# Prueba de Texto a SQL

In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT",""),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name    = os.getenv('DEPLOYMENT_NAME')

In [2]:
user_msg = """### Tablas de Postgres SQL, con sus propiedades:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### Haz una consulta para listar los nombres de los departamentos que emplearon a más de 10 empleados en los últimos 3 meses.
consulta:
"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "Eres un asistente útil."},
        {"role": "user", "content": user_msg},
    ],
  temperature=0,
  max_tokens=500
)

print(response.choices[0].message.content)

Para listar los nombres de los departamentos que emplearon a más de 10 empleados en los últimos 3 meses, puedes usar la siguiente consulta SQL:

```sql
SELECT d.name AS department_name
FROM Department d
JOIN Employee e ON d.id = e.department_id
WHERE e.id IN (
    SELECT DISTINCT employee_id
    FROM Salary_Payments
    WHERE date >= CURRENT_DATE - INTERVAL '3 months'
)
GROUP BY d.id, d.name
HAVING COUNT(e.id) > 10;
```

### Explicación:
1. **Subconsulta en `Salary_Payments`**:
   - Filtra los empleados que recibieron pagos salariales en los últimos 3 meses (`date >= CURRENT_DATE - INTERVAL '3 months'`).
   - Usa `DISTINCT` para asegurarse de considerar solo empleados únicos.

2. **Unión con `Employee` y `Department`**:
   - Une la tabla `Employee` con la tabla `Department` para asociar a los empleados con sus respectivos departamentos.

3. **Agrupación y conteo**:
   - Agrupa los resultados por departamento (`GROUP BY d.id, d.name`).
   - Usa `HAVING COUNT(e.id) > 10` para filtrar los

# Prueba de SQL a Texto

In [3]:
user_msg = """### ¿Puedes explicar qué hace este código? 
# Código: 
SELECT d.name
FROM Department d
WHERE d.id IN (
    SELECT e.department_id
    FROM Employee e
    WHERE e.id IN (
        SELECT sp.employee_id
        FROM Salary_Payments sp
        WHERE sp.date >= current_date - interval '3 months'
    )
    GROUP BY e.department_id
    HAVING COUNT(e.id) > 10
)
# Respuesta: 
"""
   
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "Eres un asistente útil."},
        {"role": "user", "content": user_msg},
    ],
   max_tokens=250)

print(response.choices[0].message.content)

Este código SQL está analizando y consultando datos de tres tablas: `Department`, `Employee` y `Salary_Payments`. Aquí tienes un desglose de lo que hace el código:

### Objetivo:
La consulta recupera el `nombre` de **departamentos** (`Department.d.name`) que cumplen con los siguientes criterios:
- **Al menos 10 empleados** en el departamento han recibido un **pago salarial** en los **últimos 3 meses**.
  
### Desglose de la Consulta:
#### 1. **Consulta principal:**
```sql
SELECT d.name
FROM Department d
WHERE d.id IN (
    ...
)
```
- Esta parte extrae los nombres de los departamentos (`d.name`) de la tabla `Department`.
- La condición es que el `id` del departamento debe coincidir con los resultados de la **subconsulta** dentro de la cláusula `WHERE`.

---

#### 2. **Primera subconsulta:**
```sql
SELECT e.department_id
FROM Employee e
WHERE e.id IN (
    ...
)
GROUP BY e.department_id
HAVING COUNT(e.id) > 10
```
- Esta subconsulta trabaja con la tabla `Employee`.
- Agrupa a los emplea